In [1]:
#####Import and install modules needed for all three parts of the assignment.


!pip install bs4
from bs4 import BeautifulSoup # this module helps in web scrapping.

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

print('Libraries imported.')


Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::ibm-wsrt-py37main-main==custom=2020
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=2020
  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
done

# All requested packages already installed.

Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::ibm-wsrt-py37main-main==custom=2020
  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=2020
  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37hafa7651_0
done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0          

In [2]:
#####Part 1 of the assignment begins here.#####


#Scrape Toronto postal code wiki
wiki_data=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
toronto_soup=BeautifulSoup(wiki_data, "html5lib")
#print(toronto_soup.prettify())

#ASSUMPTIONS: Wiki data is correct. Possible action: cross-reference with official source.

In [3]:
#Create a dataframe from the table in the Wiki soup.

table_contents=[]
table=toronto_soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
neighborhoods_df=pd.DataFrame(table_contents)
neighborhoods_df['Borough']=neighborhoods_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

neighborhoods_df.head()



,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [4]:
#Display number of rows and columns in the dataframe.

neighborhoods_df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [5]:
######Part 2 of the assignment begins here.######

#Create dataframe from Geospacial csv

geo_data_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv",index_col='Postal Code')

geo_data_df.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [6]:
#Add Latitude and Longitude to neighborhoods dataframe based on postal code

neighborhood_data_df = neighborhoods_df.join(geo_data_df, on='Postal Code') 

neighborhood_data_df.head()


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [ ]:
######Part 3 of the assignment begins here.

##Analysis is in the very last cell.


 

In [7]:
# create map of Toronto using latitude and longitude values

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)


# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhood_data_df['Latitude'], neighborhood_data_df['Longitude'], neighborhood_data_df['Borough'], neighborhood_data_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
    
map_toronto

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [8]:
#Define Foursquare credentials

CLIENT_ID = 'AD3VDOPBUDVA33LD4LROSAWTG2XBUGJUXUBLVJ0PBFWIITF3'
CLIENT_SECRET = '5VUJQRYOEDCQELVD3YGOOA25U3QRO1NUIXYL5UELAJI1PA1X' 
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AD3VDOPBUDVA33LD4LROSAWTG2XBUGJUXUBLVJ0PBFWIITF3
CLIENT_SECRET:5VUJQRYOEDCQELVD3YGOOA25U3QRO1NUIXYL5UELAJI1PA1X


In [9]:
#Create url for Get request

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=AD3VDOPBUDVA33LD4LROSAWTG2XBUGJUXUBLVJ0PBFWIITF3&client_secret=5VUJQRYOEDCQELVD3YGOOA25U3QRO1NUIXYL5UELAJI1PA1X&v=20180605&ll=43.6534817,-79.3839347&radius=500&limit=100'

In [10]:
results = requests.get(url).json()
#results

In [11]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']



In [12]:
#Create dataframe of Foursquare request results

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Japango,Sushi Restaurant,43.655268,-79.385165
3,Indigo,Bookstore,43.653515,-79.380696
4,Poke Guys,Poke Place,43.654895,-79.385052


In [17]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

print('There are {} unique categories.'.format(len(nearby_venues['categories'].unique())))

74 venues were returned by Foursquare.
There are 57 unique categories.


In [169]:
#One hot encoding for venues

# one hot encoding
toronto_onehot = pd.get_dummies(nearby_venues[['categories']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = neighborhood_data_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[33]] + list(toronto_onehot.columns[0:33])
fixed_columns = fixed_columns + list(toronto_onehot.columns[34:])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()



,Neighborhood,American Restaurant,Art Museum,Bank,Bookstore,Breakfast Spot,Bubble Tea Shop,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Department Store,Dessert Shop,Diner,Electronics Store,Fast Food Restaurant,Food & Drink Shop,Furniture / Home Store,Gastropub,Gym / Fitness Center,Hotel,Latin American Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Pizza Place,Plaza,Poke Place,Record Shop,Restaurant,Salad Place,Seafood Restaurant,Shoe Store,Shopping Mall,Smoothie Shop,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Tanning Salon,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Victoria Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,"Lawrence Manor, Lawrence Heights",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Ontario Provincial Government,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [170]:

toronto_onehot.shape

(74, 57)

In [171]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,American Restaurant,Art Museum,Bank,Bookstore,Breakfast Spot,Bubble Tea Shop,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Department Store,Dessert Shop,Diner,Electronics Store,Fast Food Restaurant,Food & Drink Shop,Furniture / Home Store,Gastropub,Gym / Fitness Center,Hotel,Latin American Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Pizza Place,Plaza,Poke Place,Record Shop,Restaurant,Salad Place,Seafood Restaurant,Shoe Store,Shopping Mall,Smoothie Shop,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Tanning Salon,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant
0,"Bathurst Manor, Wilson Heights, Downsview North",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Bayview Village,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Bedford Park, Lawrence Manor East",0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Berczy Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,"Birch Cliff, Cliffside West",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,"Brockton, Parkdale Village, Exhibition Place",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,Caledonia-Fairbanks,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Cedarbrae,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,Central Bay Street,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
9,Christie,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [172]:
#Identify most common venues for each neighborhood.

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
                 venue  freq
0           Restaurant   1.0
1  American Restaurant   0.0
2           Art Museum   0.0
3  Monument / Landmark   0.0
4        Movie Theater   0.0


----Bayview Village----
                 venue  freq
0       Clothing Store   1.0
1  American Restaurant   0.0
2          Salad Place   0.0
3        Movie Theater   0.0
4          Music Venue   0.0


----Bedford Park, Lawrence Manor East----
                  venue  freq
0  Colombian Restaurant   1.0
1   American Restaurant   0.0
2           Salad Place   0.0
3         Movie Theater   0.0
4           Music Venue   0.0


----Berczy Park----
                 venue  freq
0   Seafood Restaurant   1.0
1  American Restaurant   0.0
2           Restaurant   0.0
3  Monument / Landmark   0.0
4        Movie Theater   0.0


----Birch Cliff, Cliffside West----
                 venue  freq
0          Coffee Shop   1.0
1  American Restaurant   0.0
2          Salad Place   

In [173]:
#Function to return most common venues into neighborhood dataframe.  Called below.

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [181]:
#Create new dataframe by adding most common venues to neighborhoods.
#Must rerun this cell after k-means clustering to add cluster labels.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
    
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)  #Added to update cluster labels after running k-means clustering.
    
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,"Bathurst Manor, Wilson Heights, Downsview North",Restaurant,American Restaurant,Art Museum,Monument / Landmark,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House
1,1,Bayview Village,Clothing Store,American Restaurant,Salad Place,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Pizza Place
2,0,"Bedford Park, Lawrence Manor East",Colombian Restaurant,American Restaurant,Salad Place,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Pizza Place
3,0,Berczy Park,Seafood Restaurant,American Restaurant,Restaurant,Monument / Landmark,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House
4,2,"Birch Cliff, Cliffside West",Coffee Shop,American Restaurant,Salad Place,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Pizza Place


In [180]:
#####4. Cluster Neighborhoods

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([0, 1, 0, 0, 2, 0, 0, 0, 4, 0], dtype=int32)

In [182]:
#Join venue information dataframe with neighborhood dataframe.

# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)   Commented due to this code needing to be run in a prior cell.

toronto_merged = neighborhood_data_df

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood',how='right')

toronto_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.754328,-79.442259,0,Restaurant,American Restaurant,Art Museum,Monument / Landmark,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House
39,M2K,North York,Bayview Village,43.786947,-79.385975,1,Clothing Store,American Restaurant,Salad Place,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Pizza Place
55,M5M,North York,"Bedford Park, Lawrence Manor East",43.733283,-79.419750,0,Colombian Restaurant,American Restaurant,Salad Place,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Pizza Place
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Seafood Restaurant,American Restaurant,Restaurant,Monument / Landmark,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House
58,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,2,Coffee Shop,American Restaurant,Salad Place,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Pizza Place


In [183]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
##5. Examine Clusters

#The cells below were used to iterate different cluster results before setting on kclusters = 5.

In [184]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,North York,0,Restaurant,American Restaurant,Art Museum,Monument / Landmark,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House
55,North York,0,Colombian Restaurant,American Restaurant,Salad Place,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Pizza Place
20,Downtown Toronto,0,Seafood Restaurant,American Restaurant,Restaurant,Monument / Landmark,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House
43,West Toronto,0,Bank,American Restaurant,Salad Place,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Pizza Place
21,York,0,Chinese Restaurant,American Restaurant,Salad Place,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Pizza Place
26,Scarborough,0,American Restaurant,Art Museum,Monument / Landmark,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Pizza Place
25,Downtown Toronto,0,Furniture / Home Store,American Restaurant,Modern European Restaurant,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Pizza Place
51,Scarborough,0,Food & Drink Shop,American Restaurant,Modern European Restaurant,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Pizza Place
48,Downtown Toronto,0,Smoothie Shop,American Restaurant,Restaurant,Monument / Landmark,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House
56,York,0,Office,American Restaurant,Art Museum,Monument / Landmark,Movie Theater,Music Venue,New American Restaurant,Noodle House,Opera House,Pizza Place


In [185]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
39,North York,1,Clothing Store,American Restaurant,Salad Place,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Pizza Place
67,Central Toronto,1,Clothing Store,American Restaurant,Salad Place,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Pizza Place
13,North York,1,Clothing Store,American Restaurant,Salad Place,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Pizza Place
50,North York,1,Clothing Store,American Restaurant,Salad Place,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Pizza Place
47,East Toronto,1,Clothing Store,American Restaurant,Salad Place,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Pizza Place
15,Downtown Toronto,1,Clothing Store,American Restaurant,Salad Place,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Pizza Place
35,East York/East Toronto,1,Clothing Store,American Restaurant,Salad Place,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Pizza Place


In [186]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
58,Scarborough,2,Coffee Shop,American Restaurant,Salad Place,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Pizza Place
10,North York,2,Coffee Shop,American Restaurant,Salad Place,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Pizza Place
73,Central Toronto,2,Coffee Shop,American Restaurant,Salad Place,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Pizza Place
19,East Toronto,2,Coffee Shop,American Restaurant,Salad Place,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Pizza Place
71,Scarborough,2,Coffee Shop,American Restaurant,Salad Place,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Pizza Place


In [187]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,East York,3,Electronics Store,American Restaurant,Modern European Restaurant,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Pizza Place
30,Downtown Toronto,3,Electronics Store,American Restaurant,Modern European Restaurant,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House,Pizza Place


In [188]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,Downtown Toronto,4,Theater,American Restaurant,Restaurant,Monument / Landmark,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House
18,Scarborough,4,Theater,American Restaurant,Restaurant,Monument / Landmark,Movie Theater,Music Venue,New American Restaurant,Noodle House,Office,Opera House


In [189]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [109]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [190]:
###ANALYSIS###

# To determine neighbrhood clusters, I tried K values of 3-6 and consistently identified three clusters of most common venues: Variety, Clothing, and Coffee.
#    
# In each iteration, American Restaurant was the 2nd-most-common venue in almost every neighborhood. So I removed that venue type from the data and reran,
#  and found the same venue clusters, except Art Museum was now consistently 2nd-most-common.  So I removed that and reran, getting the same results except with Bank
#  as the new 2nd-most-common.
#
# I therefore concluded that the most relevant data to a cluster is the most common venue only, and returned the data to the orginal complete set.
# Finally, I settled on 5 clusters to include two additional categories of Electronics and Theaters, while avoiding any clusters having only one data point.